In [1]:
import spacy
from tqdm import tqdm

nlp = spacy.load('en')

In [2]:
file_type = 'test'
source_path = '/home/ml/cadencao/XSum/fairseq_files/{}.source'.format(file_type)
target_path = '/home/ml/cadencao/XSum/fairseq_files/{}.target'.format(file_type)

In [3]:
def read_lines(file_path):
    files = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            files.append(line.strip())
    return files

#### Read File

In [4]:
source = read_lines(source_path)
target = read_lines(target_path)

In [5]:
print(len(source))
assert len(source) == len(target)

11301


#### Processing

In [6]:
tp = nlp(target[0])

In [7]:
target_processed = []

for s, t in tqdm(zip(source, target)):
    t_processed = t
    t_ents = [e.text for e in nlp(t).ents]
    for e in t_ents:
        e_parts = e.split()
        for ep in e_parts:
            if ep not in s:
                if ep + ' ' in t_processed:
                    t_processed = t_processed.replace(ep + ' ', '')
                elif ' ' + ep in t_processed:
                    t_processed = t_processed.replace(' ' + ep, '')
                elif ep in t_processed:
                    t_processed = t_processed.replace(ep, '')
    # add processed
    target_processed.append(t_processed)

11301it [01:27, 128.78it/s]


In [8]:
assert len(target) == len(target_processed)

In [9]:
output_file = '{}_removed.target'.format(file_type)

with open(output_file, 'w') as f:
    for t in target_processed:
        f.write(t + '\n')